In [34]:
import os
import re

import pandas as pd
import numpy as np

### Data Sources and Legend

- https://satudata.jakarta.go.id/
- https://www.kaggle.com/datasets/senadu34/air-quality-index-in-jakarta-2010-2021

- **date** : Date of air quality measurement
- **station** : Measurement location at the station
- **pm10** : Particulate matter one of the measured parameters
- **pm25** : Particulate matter one of the parameters measured
- **so2** : Sulfide (in the form of SO2) one of the parameters measured
- **co** : Carbon Monoxide one of the parameters measured
- **o3** : Ozone is one of the measured parameters
- **no2** : Nitrogen dioxide one of the parameters measured
- **max** : The highest measured value of all parameters measured at the same time
- **critical**: The parameter with the highest measurement results
- **category** : Category based on the calculation of the air pollution standard index

### Load Dataset

In [2]:
base_path = r"F:\Data\datas\Air Quality"

In [3]:
ISPU_JKT_2022_filepath = os.path.join(base_path, "ISPU DKI 2022.csv")
ISPU_JKT_2023_filepath = os.path.join(base_path, "ISPU DKI 2023.csv")
ISPU_JKT_2024_filepath = os.path.join(base_path, "ISPU DKI 2024.csv")

df_ispa_dki_2022 =  pd.read_csv(ISPU_JKT_2022_filepath)
df_ispa_dki_2023 =  pd.read_csv(ISPU_JKT_2023_filepath)
df_ispa_dki_2024 =  pd.read_csv(ISPU_JKT_2024_filepath)

In [4]:
paths = [os.path.join(base_path, f"ispu DKI\ispu_dki{i}.csv") for i in range(1, 6)]

df_ispa_dki = pd.DataFrame()
for path in paths:
    if df_ispa_dki.empty:
        df_ispa_dki = pd.read_csv(path)
    else:
        df_ispa_dki = pd.concat([df_ispa_dki, pd.read_csv(path)], axis=0)

### Data Cleaning (Individually)

#### Remap column names

In [5]:
remap_column_names = {
    'tanggal':'date',
    'pm_sepuluh':'pm10', 
    'pm_duakomalima':'pm25',
    'sulfur_dioksida':'so2',
    'karbon_monoksida':'co',
    'ozon':'o3',
    'nitrogen_dioksida':'no2',
    'parameter_pencemar_kritis':'critical',
    'categori':'category',
    'kategori':'category',
    'stasiun':'station',
    'lokasi_spku':'station'    
}

In [6]:
df_ispa_dki_2022.rename(remap_column_names, axis=1, inplace=True)
df_ispa_dki_2023.rename(remap_column_names, axis=1, inplace=True)
df_ispa_dki_2024.rename(remap_column_names, axis=1, inplace=True)
df_ispa_dki.rename(remap_column_names, axis=1, inplace=True)

#### Fixing row-value error

In [7]:
station_mode_dki_2022 = df_ispa_dki_2022['station'].mode().iloc[0]
df_ispa_dki_2022['station'].replace('0', station_mode_dki_2022, inplace=True)

In [8]:
# Assuming the error value is '12/31/2022' as the missing date left on that day
df_ispa_dki_2022.at[0, 'date'] = '12/31/2022'

#### Fixing column-date error

In [9]:
df_ispa_dki_2024 = df_ispa_dki_2024.assign(
    date = lambda df: "2024" + '-' + df['bulan'].astype(str) + '-' + df['date'].astype(str)
)

#### Convert "date" to date time date type

In [10]:
df_ispa_dki_2022['date'] = pd.to_datetime(df_ispa_dki_2022['date'])
df_ispa_dki_2023['date'] = pd.to_datetime(df_ispa_dki_2023['date'])
df_ispa_dki_2024['date'] = pd.to_datetime(df_ispa_dki_2024['date'])
df_ispa_dki['date'] = pd.to_datetime(df_ispa_dki['date'])

### Join Dataset

In [11]:
df_ispa_dki_all = pd.concat([
    df_ispa_dki,
    df_ispa_dki_2022,
    df_ispa_dki_2023,
    df_ispa_dki_2024,
])

df_ispa_dki_all = df_ispa_dki_all.reset_index()

### Feature Selection

In [12]:
COLUMNS = ['date', 'station', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max', 'critical', 'category']

In [13]:
df_ispa_dki_all = df_ispa_dki_all[COLUMNS].copy()

### Data Cleaning (Generally)

#### Handling Numerical-Features Error

In [14]:
FEATURES = ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2']

In [15]:
# error check
_df_ispa_dki_all = df_ispa_dki_all.copy()

In [16]:
error_report = set()
for feature in FEATURES:
    pure_nan_idx = np.argwhere(df_ispa_dki_all[feature].isna()).ravel()
    df_ispa_dki_all[feature] = pd.to_numeric(df_ispa_dki_all[feature], errors='coerce')
    
    # Coerce will change the alphabetic string value to nan
    error_nan_idx = np.argwhere(df_ispa_dki_all[feature].isna()).ravel()    
    error_report.update(set(error_nan_idx).difference(pure_nan_idx))

In [17]:
_df_ispa_dki_all.loc[list(error_report)][FEATURES].melt()['value'].unique()

array(['54', '15', '48', '70', '-', '49', '52', '53', '55', '36', '28',
       '18', '20', '22', '58', '37', '24', '32', '33', '23', '50', '42',
       '21', '29', '30', '51', '40', '25', '26', '35', '17', '27', '34',
       '19', '38', '16', '13', '47', '39', '44', '31', '45', '59', '76',
       '57', '46', '60', '61', '66', '68', '63', '43', '65', '56', '41',
       '62', '67', '64', '81', '107', '83', '73', '74', '111', '135',
       '106', '113', '92', '82', '71', '88', '86', '123', '109', '89',
       '78', '102', '104', '114', '119', '118', '97', '115', '170', '136',
       '139', '126', '128', '110', '91', '85', '72', '95', '99', '79',
       '80', '98', '133', '121', '108', '84', '112', '93', '77', '151',
       '120', '90', '101', '130', '125', '146', '117', '124', '129',
       '116', '105', '96', '134', '217', '75', '103', '122', '138', '127',
       '14', '4', '7', '5', '3', '2', '9', '8', '6', '10', '11', '12',
       '---', '1'], dtype=object)

In [18]:
df_ispa_dki_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24865 entries, 0 to 24864
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      24865 non-null  datetime64[ns]
 1   station   24865 non-null  object        
 2   pm10      21130 non-null  float64       
 3   pm25      4372 non-null   float64       
 4   so2       21873 non-null  float64       
 5   co        22041 non-null  float64       
 6   o3        21806 non-null  float64       
 7   no2       21877 non-null  float64       
 8   max       24832 non-null  object        
 9   critical  22448 non-null  object        
 10  category  24865 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 2.1+ MB


#### Handling Missing Values

In [19]:
non_na_idx = df_ispa_dki_all[FEATURES].dropna(axis=0, how='all').index
na_idx = list(set(df_ispa_dki_all.index).difference(non_na_idx))
na_based_features = df_ispa_dki_all.loc[na_idx, :].shape[0]
print(f"Num. of missing values based on features: {na_based_features}")

Num. of missing values based on features: 2403


In [20]:
dropped_idx = df_ispa_dki_all.query("category == 'TIDAK ADA DATA'").index
na_based_category = df_ispa_dki_all.loc[dropped_idx, :].shape[0]
print(f"Num. of missing values based on 'TIDAK ADA DATA' category: {na_based_features}")

Num. of missing values based on 'TIDAK ADA DATA' category: 2403


In [21]:
print("Before dropped:", df_ispa_dki_all.shape)
df_ispa_dki_all.drop(index=dropped_idx, inplace=True)
print("After dropped:", df_ispa_dki_all.shape)

Before dropped: (24865, 11)
After dropped: (22461, 11)


#### Remap station names

In [22]:
remap_station_names = {    
    "DKI1": "DKI1 (Bunderan HI)",
    "DKI1 Bunderan HI": "DKI1 (Bunderan HI)",
    "DKI1 Bundaran Hotel Indonesia (HI)": "DKI1 (Bunderan HI)",
    "DKI2": "DKI2 (Kelapa Gading)",
    "DKI2 Kelapa Gading": "DKI2 (Kelapa Gading)",
    "DKI3": "DKI3 (Jagakarsa)",
    "DKI3 Jagakarsa": "DKI3 (Jagakarsa)",
    "DKI4": "DKI4 (Lubang Buaya)",
    "DKI4 Lubang Buaya": "DKI4 (Lubang Buaya)",
    "DKI5": "DKI5 (Kebon Jeruk)",
    "DKI5 Kebon Jeruk Jakarta Barat": "DKI5 (Kebon Jeruk)",
    "DKI5 Kebon Jeruk": "DKI5 (Kebon Jeruk)",
}

In [23]:
df_ispa_dki_all['station'].value_counts()

station
DKI1 (Bunderan HI)                    4273
DKI2 (Kelapa Gading)                  4032
DKI3 (Jagakarsa)                      3980
DKI4 (Lubang Buaya)                   3974
DKI5 (Kebon Jeruk)                    3277
DKI2 Kelapa Gading                     514
DKI3 Jagakarsa                         513
DKI4 Lubang Buaya                      510
DKI1 Bunderan HI                       451
DKI5 Kebon Jeruk                       303
DKI4                                   267
DKI5 Kebon Jeruk Jakarta Barat         207
DKI1 Bundaran Hotel Indonesia (HI)      62
DKI2                                    39
DKI3                                    33
DKI5                                    22
DKI1                                     4
Name: count, dtype: int64

In [24]:
df_ispa_dki_all['station'].replace(remap_station_names, inplace=True)

In [25]:
df_ispa_dki_all['station'].value_counts()

station
DKI1 (Bunderan HI)      4790
DKI4 (Lubang Buaya)     4751
DKI2 (Kelapa Gading)    4585
DKI3 (Jagakarsa)        4526
DKI5 (Kebon Jeruk)      3809
Name: count, dtype: int64

#### Remap critical names

In [26]:
df_ispa_dki_all['critical'].value_counts()

critical
O3       10868
PM10      6258
PM25      3820
CO         576
SO2        424
PM2,5      333
2          112
1           30
NO2          9
3            7
5            3
Name: count, dtype: int64

In [27]:
df_ispa_dki_all['critical'] = df_ispa_dki_all[FEATURES].idxmax(axis=1).str.upper()

In [28]:
df_ispa_dki_all['critical'].value_counts()

critical
O3      11101
PM10     6345
PM25     3978
SO2       550
CO        439
NO2        48
Name: count, dtype: int64

### Feature Engineering

#### Date Features

In [29]:
df_ispa_dki_all['year'] = df_ispa_dki_all['date'].dt.year
df_ispa_dki_all['quarter'] = df_ispa_dki_all['date'].dt.quarter
df_ispa_dki_all['month'] = df_ispa_dki_all['date'].dt.month_name().map(lambda x: x[:3])
df_ispa_dki_all['day'] = df_ispa_dki_all['date'].dt.day
df_ispa_dki_all['day_of_week'] = df_ispa_dki_all['date'].dt.day_name().map(lambda x: x[:3])

In [32]:
df_ispa_dki_all['station'].value_counts()

station
DKI1 (Bunderan HI)      4790
DKI4 (Lubang Buaya)     4751
DKI2 (Kelapa Gading)    4585
DKI3 (Jagakarsa)        4526
DKI5 (Kebon Jeruk)      3809
Name: count, dtype: int64

#### Geo-Administration Features

In [51]:
get_district = {
    'DKI1': 'Jakarta Pusat', 
    'DKI2': 'Jakarta Utara', 
    'DKI3': 'Jakarta Selatan',
    'DKI4': 'Jakarta Timur',
    'DKI5': 'Jakarta Barat'
}

get_sub_district = {
    'Bunderan HI': 'Menteng', 
    'Lubang Buaya': 'Cipayung', 
    'Kelapa Gading': 'Kelapa Gading',
    'Jagakarsa': 'Jagakarsa',
    'Kebon Jeruk': 'Kebonjeruk'
}

In [71]:
pattern = r'\((.*?)\)'

df_ispa_dki_all = df_ispa_dki_all.assign(
    district = lambda df: df['station'].map(lambda x: get_district[x[:4]]),
    sub_district = lambda df: df['station'].str.extract(pattern).map(lambda x: get_sub_district[x])
)

### Save the cleaned data

In [75]:
df_ispa_dki_all = df_ispa_dki_all.sort_values("date")

In [76]:
df_ispa_dki_all.to_csv("df_ispa_dki_all.csv", index=False)